# Shortest Path Problem

The shortest path problem (SPP) is a fundamental problem in graph theory and network analysis. It involves finding the most efficient way to traverse a network from a given origin to a destination, minimizing the total cost or distance traveled.

In [ ]:
from IPython.display import Image

Image(filename="figures/spp.jpg", width=300)

In a bustling city, a determined traveler stands at point 'O'—their current location. Their destination, point 'D', lies somewhere else in the urban maze. The city streets intersect at nodes labeled 'a' to 'f'. Each connection between nodes has a specific cost—a distance to traverse or a time to spend. The traveler meticulously recorded these costs, revealing the intricate web of paths.

Armed with this map, our adventurer embarks on a quest. Their goal? To reach point 'D' while minimizing the total distance traveled. They analyze the arc costs, weigh their options, and navigate the network with precision. Can you help them find the shortest path to their destination?

## Mathematical model

### Sets and parameters

- $O$ and $D$ are the origin and destination nodes, respectively
- $N$ is the set of all nodes in the network
- $A$ is the set of all arcs in the network
- $c_{i,j}$ is the cost (distance, time, etc.) associated with traveling from node $i$ to node $j$

### Decision variables

For each arc $(i, j) \in A$ in the network, we define a decision variable $x_{i,j}$ as:
$$
x_{ij} = \begin{cases}
1 & \text{if arc $(i, j)$ is traversable} \\
0 & \text{otherwise}\\
\end{cases}
$$

### Objective function

Our goal is to minimize the total distance traveled. Therefore, the objective function is:
$$
\min \ \sum_{(i,j) \in A} c_{ij}\, x_{ij},
$$

### Constraints

Flow conservation constraints ensure that the traveler enters and exits each node exactly once (except for the start and destination nodes):
$$
\begin{align*}
    & \sum_{j: (i, j)\in A} x_{ij} = 1,\quad && i=O\\
    & \sum_{i: (i, j)\in A} x_{ij} = 1,\quad && j=D\\
    & \sum_{j: (i, j)\in A} x_{ij} = \sum_{j: (j, i)\in A} x_{ji},\quad && i\in N\setminus\{O, D\}
\end{align*}
$$

## A problem instance

In [ ]:
from IPython.display import Image

Image(filename="figures/spp-example.png", width=350)

In [ ]:
# Origin and destination nodes
O, D = "O", "D"

# Set of intermediate nodes
Intermediates = {"a", "b", "c", "d", "e", "f"}

# Set of all nodes
N = {O, D} | Intermediates

In [ ]:
# Map arcs to their cost
arc_cost = dict(
    {
        ("O", "a"): 6,
        ("O", "b"): 2,
        ("O", "c"): 2,
        ("a", "d"): 5,
        ("a", "e"): 4,
        ("b", "e"): 4,
        ("c", "f"): 1,
        ("d", "D"): 1,
        ("f", "e"): 2,
        ("e", "D"): 3,
        ("f", "D"): 6,
    }
)

# Set of arcs
A = set(arc_cost.keys())
print(A)

## The concrete Pyomo model

In [ ]:
!pip install gurobipy pyomo

import pyomo.environ as pyo
from pyomo.opt import SolverFactory

# Include your WSL license information
solver_options = {}

In [ ]:
# Create the model
mod = pyo.ConcreteModel(name="SPP")

# Decision variables
mod.x = pyo.Var(A, domain=pyo.Binary, name="arc")

# Objective function
mod.obj = pyo.Objective(
    expr=sum(arc_cost[arc] * mod.x[arc] for arc in A),
    sense=pyo.minimize,
)

# Flow conservation constraints:
mod.balance = pyo.ConstraintList()
for i in N:
    if i == O:
        expr = sum(mod.x[arc] for arc in A if arc[0] == i)
        mod.balance.add(expr == 1)
    elif i == D:
        expr = sum(mod.x[arc] for arc in A if arc[1] == i)
        mod.balance.add(expr == 1)
    else:
        expr1 = sum(mod.x[arc] for arc in A if arc[0] == i)
        expr2 = sum(mod.x[arc] for arc in A if arc[1] == i)
        mod.balance.add(expr1 == expr2)

In [ ]:
# Call the solve and solve the model
opt = SolverFactory("gurobi", solver_io="python", manage_env=True, solver_options=solver_options)
results = opt.solve(mod, tee=True)

In [ ]:
# Print the selected arcs
for arc in A:
    if pyo.value(mod.x[arc]) > 0.5:
        print(arc)

# Print the total cost
print("Total cost = ", pyo.value(mod.obj))

### Exercise
Print the path from the origin node to the destination node. 

In [ ]:
path = [O]
while path[-1] != D:
    for arc in A:
        if arc[0] == path[-1] and pyo.value(mod.x[arc]) > 0.5:
            path.append(arc[1])
            break

print(path)